In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.vllm import start_vllm_server, vllm_server_metrics

model = "NousResearch/Hermes-2-Theta-Llama-3-8B"

shutdown_server, client = await start_vllm_server(
    disable_log_requests=True,
    model=model,
)

/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
/home/ubuntu/atreides/.venv/lib/python3.12/site-packages/vllm/connections.py:8: RuntimeWarning: Failed to read commit hash:
No module named 'vllm._version'
  from vllm.version import __version__ as VLLM_VERSION
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]
Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:01,  1.56it/s]
Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.30it/s]
Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:01<00:00,  1.69it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.56it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.54it/s]



In [26]:
shutdown_server()

False

In [ ]:
from lib.rl.sampler import CompletionSampler

completion_sampler = CompletionSampler(
    client,
)

In [ ]:
import asyncio
from lib.rl.completion import Completion
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from typing import Callable, Coroutine


class Episode:
    def __init__(
        self,
        messages: list[ChatCompletionMessageParam],
        on_sample: Callable[[list[Completion]], None | Coroutine[None, None, None]],
        get_easier_episode: (
            Callable[[], "Episode" | Coroutine[None, None, None]] | None
        ) = None,
        get_harder_episode: (
            Callable[[], "Episode" | Coroutine[None, None, None]] | None
        ) = None,
    ) -> None:
        self.completion = Completion(messages=messages)  # type: ignore
        self.on_sample = on_sample
        self.get_easier_episode = get_easier_episode
        self.get_harder_episode = get_harder_episode
        self.weight = 1.0
        self.task = asyncio.create_task(asyncio.sleep(0))

In [ ]:
async def on_sample(completions) -> None: ...


episode = Episode(
    messages=[
        {"role": "user", "content": "Hello, how are you doing today?"},
    ],
    on_sample=on_sample,
)

In [ ]:
def get_episode() -> Episode:
    ...

In [ ]:
import inspect

branch_factor = 2
min_requests = 10
abs_buffer_size = 100
weighted_buffer_size = 200
buffer: set[Episode] = set()


async def sample_completions(episode: Episode) -> None:
    if episode.completion.children:
        parent = episode.completion
    else:
        parent = max(
            episode.completion.descendants(),
            key=lambda c: abs(c.advantage()),
        )
    completions = await completion_sampler.sample_completions(
        parent,
        n=branch_factor,
    )
    on_sample = episode.on_sample(completions)
    if inspect.iscoroutine(on_sample):
        await on_sample
    if all(c.advantage() == 0 for c in episode.completion.descendants()):
        buffer.remove(episode)


while True:
    await asyncio.sleep(1)
    running, pending = vllm_server_metrics()
    for _ in range(0, running - pending + min_requests, branch_factor * 2):
        if (
            len(buffer) < abs_buffer_size
            or sum(e.weight for e in buffer) < weighted_buffer_size
        ):
            episode = get_episode()
            buffer.add(episode)
            episode.task = asyncio.create_task(sample_completions(episode))